In [1]:
!pip install evaluate
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 3.8 MB/s eta 0:00:00


In [1]:
import math

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from transformers import AutoTokenizer, MT5ForConditionalGeneration, set_seed
# import evaluate

from tqdm import tqdm

set_seed(42)

#bleu = evaluate.load('sacrebleu')

In [3]:
class TData(Dataset):
    def __init__(self, df, tokenizer, device='cpu'):
        super(TData, self).__init__()

        self.df = df
        self.tokenizer = tokenizer
        self.device= device

    def __getitem__(self, i):
        row = self.df.iloc[i]

        cloze = f'answer: {row["answer"]} context: {row["cloze"]}'
        question = f'question: {row["question"]}'
        
        encoder_inputs = self.tokenizer(cloze, padding='max_length', max_length=256,
                                        truncation=True, return_tensors='pt')
        decoder_outputs = self.tokenizer(question, padding='max_length', max_length=256,
                                         truncation=True, return_tensors="pt")

        encoder_inputs = {k: v[0].to(self.device) for k, v in encoder_inputs.items()}
        decoder_outputs = {f'decoder_{k}': v[0].to(self.device) for k, v in decoder_outputs.items()}

        return {**encoder_inputs, **decoder_outputs, 'labels': decoder_outputs['decoder_input_ids']}

    def __len__(self):
        return len(self.df)


In [20]:
def train_step(model, train_loader, optimizer):
    model.train()

    losses, ppls, bleus = [], [], []
    pbar = tqdm(train_loader)
    for i, batch in enumerate(pbar):
        optimizer.zero_grad()
        out = model(**batch)
        loss = out.loss

        loss.backward()
        optimizer.step()

        losses.append(loss.item())
        ppls.append(math.exp(loss.item()))
        
        references = [[o] for o in tokenizer.batch_decode(batch['decoder_input_ids'], skip_special_tokens=True)]
        predictions = tokenizer.batch_decode(out.logits.argmax(dim=-1), skip_special_tokens=True)
        results = bleu.compute(predictions=predictions, references=references)
        bleus.append(results['score'])

        pbar.set_description(f'Batch {i+1}/{len(train_loader)}: Loss: {np.mean(losses):.4f} - Perplexity: {np.mean(ppls):.4f} - Bleu: {np.mean(bleus):.4f}')

    return np.mean(losses), np.mean(ppls), np.mean(bleus)


def eval_step(model, val_loader):
    model.eval()

    losses, ppls, bleus = [], [], []
    with torch.no_grad():
        pbar = tqdm(val_loader)
        for i, batch in enumerate(pbar):
            out = model(**batch)
            loss = out.loss           
            
            losses.append(loss.item())
            ppls.append(math.exp(loss.item()))
            
            references = [[o] for o in tokenizer.batch_decode(batch['decoder_input_ids'], skip_special_tokens=True)]
            predictions = tokenizer.batch_decode(out.logits.argmax(dim=-1), skip_special_tokens=True)
            results = bleu.compute(predictions=predictions, references=references)
            bleus.append(results['score'])
            
            pbar.set_description(f'Batch {i+1}/{len(val_loader)}: Loss: {np.mean(losses):.4f} - Perplexity: {np.mean(ppls):.4f} - Bleu: {np.mean(bleus):.4f}')
        
            if i == 0:
                print('Example Reference: ', references[0])
                print('Example Prediction: ', predictions[0])

    return np.mean(losses), np.mean(ppls), np.mean(bleus)


def train(model, train_loader, optimizer, val_loader=None, device='cpu'):
    train_losses, train_ppls, train_bleus = [], [], []
    val_losses, val_ppls, val_bleus = [], [], []

    for epoch in range(EPOCHS):
        print(f'Epoch {epoch+1}/{EPOCHS}')

        train_loss, train_ppl, train_bleu = train_step(model, train_loader, optimizer)
        train_losses.append(train_loss)
        train_ppls.append(train_ppl)
        train_bleus.append(train_bleu)

        if val_loader is not None:
            val_loss, val_ppl, val_bleu = eval_step(model, val_loader)
            val_losses.append(val_loss)
            val_ppls.append(val_ppl)
            val_bleus.append(val_bleu)
        
        test_case = val_df.iloc[0]
        question = ask(model, test_case.answer, test_case.cloze, device=DEVICE)
        print(f'GENERATED -> {question}')

    return train_losses, train_ppls, train_bleus, val_losses, val_ppls, val_bleus

@torch.no_grad()
def ask(model, answer, cloze, device='cpu'):
    encoder_inputs = tokenizer(f'answer: {answer} context: {cloze}', return_tensors='pt', truncation=True).input_ids.to(device)
    decoder_inputs = tokenizer('question: ', return_tensors='pt', add_special_tokens=False).input_ids.to(device)
    
    """
    generated_ids = model.generate(encoder_inputs,
                                   decoder_input_ids=decoder_inputs,
                                   pad_token_id=tokenizer.eos_token_id,
                                   num_beams=5, do_sample=True,
                                   top_k=50, top_p=0.95, early_stopping=True,
                                   no_repeat_ngram_size=1,
                                   max_new_tokens=50, num_return_sequences=1)
    """
    generated_ids = model.generate(encoder_inputs, decoder_input_ids=decoder_inputs,
                                   num_beams=1, num_return_sequences=1, 
                                   do_sample=False, max_new_tokens=50)

    return tokenizer.batch_decode(generated_ids)

ask(model, answer=answer, cloze=cloze, device='cpu')

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


["question:'''''''''''''''''''''''''"]

In [19]:
tokenizer = AutoTokenizer.from_pretrained('google/mt5-small')
model = MT5ForConditionalGeneration.from_pretrained('google/mt5-small').to('cpu')

In [11]:
cloze = '''Panthers hattında ayrıca, sadece 9 başlangıçta 5 sack eden uç çizgi savunmacısı Kony Ealy ile birlikte 136 kez ile NFL'nin aktif kariyer sack lideri ve 5 kez profesyonel bir top atıcısı olan Jared Allen öne çıkmaktadır.'''
answer = '''136'''

ask(model, answer=answer, cloze=cloze, device='cpu')

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


['question:::::::::::::::::::::::::::::::::::::::::::::::::::']

In [21]:
from transformers import pipeline
generator = pipeline('text2text-generation', model=model, tokenizer=tokenizer)
generator(f'answer: {answer} context: {cloze}', max_length=50, num_beams=1, do_sample=False)

[{'generated_text': '<extra_id_0> lideri'}]

In [5]:
TRAIN_DIR = '/kaggle/input/my-quad/my_quad.csv'

trainval_df = pd.read_csv(TRAIN_DIR)

train_df = trainval_df.sample(frac=.85, random_state=42)
val_df = trainval_df.drop(train_df.index)

In [6]:
BATCH_SIZE = 2
EPOCHS = 20
DEVICE = 'cuda'

In [7]:
model_path = 'google/mt5-small'

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = MT5ForConditionalGeneration.from_pretrained(model_path).to(DEVICE)

optimizer = optim.Adadelta(model.parameters())

/opt/conda/lib/python3.7/site-packages/transformers/convert_slow_tokenizer.py:447: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"


In [4]:
train_data = TData(train_df, tokenizer, device=DEVICE)
val_data = TData(val_df, tokenizer, device=DEVICE)

NameError: name 'TData' is not defined

In [9]:
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE)

In [10]:
ex = val_df.iloc[0]
print(f'answer -> {ex.answer}\ncloze -> {ex.cloze}\nquestion -> {ex.question}')

answer -> 136
cloze ->  Panthers hattında ayrıca, sadece 9 başlangıçta 5 sack eden uç çizgi savunmacısı Kony Ealy ile birlikte 136 kez ile NFL'nin aktif kariyer sack lideri ve 5 kez profesyonel bir top atıcısı olan Jared Allen öne çıkmaktadır.
question -> Jared Allen'ın kaç tane kariyer sack edişi vardır?


In [18]:
train_losses, train_ppls, train_bleus, val_losses, val_ppls, val_bleus = train(model, train_loader, optimizer, val_loader=val_loader, device=DEVICE)

Epoch 1/20


Batch 566/566: Loss: 1.3193 - Perplexity: 2293294902424658727927808.0000 - Bleu: 3.0374: 100%|██████████| 566/566 [01:54<00:00,  4.93it/s] 
Batch 5/100: Loss: 0.4483 - Perplexity: 1.5682 - Bleu: 6.2415:   3%|▎         | 3/100 [00:00<00:04, 21.41it/s]

Example Reference:  ["question: Jared Allen'ın kaç tane kariyer sack edişi vardır?"]
Example Prediction:  question: ne  yıli ne ne ne ne    ne?


Batch 100/100: Loss: 0.4573 - Perplexity: 1.5908 - Bleu: 5.6788: 100%|██████████| 100/100 [00:04<00:00, 20.93it/s]


GENERATED -> ['question:::::::::::::::::::::::::::::::::::::::::::::::::::']
Epoch 2/20


...

GENERATED -> ['question:::::::::::::::::::::::::::::::::::::::::::::::::::']
Epoch 20/20


Batch 566/566: Loss: 0.0040 - Perplexity: 1.0040 - Bleu: 92.6174: 100%|██████████| 566/566 [01:54<00:00,  4.95it/s]
Batch 5/100: Loss: 0.0002 - Perplexity: 1.0002 - Bleu: 100.0000:   3%|▎         | 3/100 [00:00<00:04, 21.38it/s]

Example Reference:  ["question: Jared Allen'ın kaç tane kariyer sack edişi vardır?"]
Example Prediction:  question: Jared Allen'ın kaç tane kariyer sack edişi vardır?


Batch 100/100: Loss: 0.0008 - Perplexity: 1.0008 - Bleu: 98.5489: 100%|██████████| 100/100 [00:04<00:00, 20.93it/s]


GENERATED -> ['question:::::::::::::::::::::::::::::::::::::::::::::::::::']


In [17]:
ex = val_df.iloc[0]
ask(model, ex.answer, ex.cloze, device=DEVICE)

["question:'''''''''''''''''''''''' '"]

In [19]:
model.save_pretrained('mt5_12april')

In [ ]:
!zip -r best_bert_gpt2.zip best_bert_gpt2/

In [ ]:
while True:
    pass

In [ ]:
!cp /content/attacker_bert_gpt2.pt /content/drive/MyDrive/adversarial-taboo/adversarial-taboo-models